In [ ]:
# Imports
from helpers import *
from imblearn.over_sampling import RandomOverSampler
import graphviz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

### Numpy Print Options ###
np.set_printoptions(
    threshold=2000, # 1000
    edgeitems=10, # 3
    linewidth=180, # 75
    precision=2
)

In [ ]:
dtypes = {
    'survived': np.uint8,
    'honor': 'category',
    'sex': 'category',
    'clare': np.uint8,
    'age': np.uint8,
    'fare': np.uint8,
    'cabin': 'category',
    'origin': 'category',
    'alone': 'category'
}

In [ ]:
train_raw = pd.read_csv('cleaned_train.csv', dtype=dtypes, header=0, index_col=0)
test_raw = pd.read_csv('cleaned_test.csv', dtype=dtypes, header=0, index_col=0)

In [ ]:
train = pd.get_dummies(train_raw)
test = pd.get_dummies(test_raw)
train_men = train.query('sex_male == 1')
train_women = train.query('sex_female == 1')
test_men = test.query('sex_male == 1')
test_women = test.query('sex_female == 1')
X = train.drop('survived', axis=1)
M = train_men.drop('survived', axis=1)
W = train_women.drop('survived', axis=1)
y = train['survived']
m = train_men['survived']
w = train_women['survived']

In [ ]:
pct = lambda v: int(v * 100)

def test_data(X, y, title, clf, sampler=None, splits=3):
    kfold = KFold(n_splits=splits, shuffle=True)
    avgs = []
    for train_i, test_i in kfold.split(X):
        X_train, X_test = X.iloc[train_i], X.iloc[test_i]
        y_train, y_test = y.iloc[train_i], y.iloc[test_i]
        if sampler:
            X_train, y_train = sampler.fit_resample(X_train, y_train)
        model = clf.fit(X_train, y_train)
        preds = model.predict(X_test)
        #print(' '.join('{1}{0:.10}'.format(k, int(v * 100)) for v, k in sorted(zip(model.feature_importances_, X.columns), reverse=True)[:6]))
        report = metrics.classification_report(y_test, preds, output_dict=True)
        avgs.append((metrics.accuracy_score(y_test, preds), report['0']['f1-score'], report['1']['f1-score']))
        #print(' '.join('0: {0:.1}: {1:}'.format(k, int(v * 100)) for k, v in report['0'].items()))
        #print(' '.join('1: {0:.1}: {1:}'.format(k, int(v * 100)) for k, v in report['1'].items()))
    avg = [pct(sum(lst)/3) for lst in zip(*avgs)]
    print(avg, ' ac 0f 1f', '='*5, title, '='*5)


def tests():
    test_data(X, y, 'All', abc)
    test_data(X, y, 'All Samp', abc, sampler=samp)
    test_data(X, y, 'All Weighted', wabc)
    test_data(W, w, 'Women Samp', abc, sampler=samp)
    test_data(W, w, 'Women Weighted', wabc)
    test_data(M, m, 'Men Samp', abc, sampler=samp)
    test_data(M, m, 'Men Weighted', wabc)

In [ ]:
dtc = DecisionTreeClassifier(max_depth=1, criterion='entropy', min_samples_split=2, min_samples_leaf=10, class_weight=None)
wdtc = DecisionTreeClassifier(max_depth=1, criterion='entropy', min_samples_split=2, min_samples_leaf=10, class_weight='balanced')
abc = AdaBoostClassifier(base_estimator=dtc, n_estimators=50, learning_rate=.1)
wabc = AdaBoostClassifier(base_estimator=wdtc, n_estimators=50, learning_rate=.1)
samp = RandomOverSampler(sampling_strategy='minority')

In [ ]:
tests()

In [ ]:
tests()

In [ ]:
print('split=10,leaf=1')
tests()

In [ ]:
# Output Test
X, y = samp.fit_resample(X, y)
model = abc.fit(X, y)
preds = model.predict(test)

In [ ]:
res = pd.DataFrame(preds, index=test.index, columns=['Survived'])
res.index.names = ['PassengerId']

In [ ]:
res

In [ ]:
res.to_csv('preds_ada_oversampled.csv', header=True, index=True)